In [86]:
import pandas as pd
import re
from os import listdir
from tqdm import tqdm
from anytree import Node, NodeMixin, RenderTree

In [24]:
def toRow(df, entry):
    row = {}
    row['job'] = entry['job']
    row['page'] = df.loc[entry['nodes'][-1]].page.values[0]
    row['file'] = df.loc[entry['nodes'][-1]].filename.values[0]
    levels = ['L1', 'L2', 'L3', 'L4', 'L5', 'L6']
    for i, node in enumerate(entry['nodes']):
        row[levels[i]] = ' '.join(df.loc[node].text.values)
    row['amount'] = df.loc[entry['nodes'][-1][-2]].text
    return row


def get_patern_of_bullet(String):
    regx = [
        ('\d{5}-\d+$', 70),
        ('[1-9][0-9]*(\.[1-9][0-9]*)*\)$', 20),
        ('\(\d*(\.?\d*)*\)$', 50),
        ('[1-9][0-9]*(\.[1-9][0-9]*)+$', 2),
        ('[1-9][0-9]*\.$', 1)
    ]

    for r, l in regx:
        if re.match(r, String):
            if l in [2, 20, 50]:
                l = String.count('.') + l
            return r, l
    return '', 0

In [23]:
class Budget(NodeMixin):
    def __init__(self, name, idx, bullet, parent=None, children=None):
        super(Budget, self).__init__()
        self.name = name
        self.idx = idx
        self.bullet = bullet
        self.parent = parent
        if children:
            self.children = children
    def __repr__(self):
        return self.name


In [175]:
df = pd.read_csv('bkk-bud_cleaned_reindex.csv', index_col='index')
df = df.sort_values(by=['filename', 'page', 'line_num']).reset_index(drop=True)
fpl_group = df.groupby(['filename', 'page', 'line_num'])

In [181]:
job = ''
bulletFlag = False
root = Budget('BKK2022', [], -2)
entry = []
entries = []
curr = root

for i, group in tqdm(fpl_group):
    lineText = group.text.values
    if not i[2]:
        continue

    if (lineText[0].startswith('งาน') and i[2] <= 2) or (lineText[0].startswith('งาน:')):
        entries.append(('job', group.index.to_list()))
        continue

    if ' ' in group.iloc[0].text:
        bullet = get_patern_of_bullet(group.iloc[0].text.split(' ')[0])
    else:
        bullet = get_patern_of_bullet(group.iloc[0].text)

    if lineText[0].startswith('งาน'):
        if (lineText[-1] == 'บาท'):
            lineText = lineText[:-2]
        job = ' '.join(lineText)
        continue
    elif bullet[1]:
        bulletFlag = True

    if bulletFlag:
        entry += group.index.to_list()
        if lineText[-1] == 'บาท':
            bulletFlag = False
            entries.append(('list', entry))
            entry = []


100%|██████████| 43226/43226 [00:15<00:00, 2826.16it/s]


In [183]:
root = Budget('BKK2022', [], -2)

job_root = None
curr = None

for l, entry in tqdm(entries):
  entryText = ' '.join(df.loc[entry].text.values)

  if l == 'job':
    job_root = Budget(entryText, entry, -1, root)
    curr = job_root
    continue

  entry_bullet = get_patern_of_bullet(df.loc[entry[0]].text.split(' ')[0])
  if curr == job_root:
     curr = Budget(entryText, entry, entry_bullet[1], job_root)
  else:
    while curr.bullet != job_root and entry_bullet[1] <= curr.bullet:
      curr = curr.parent
    curr = Budget(entryText, entry, entry_bullet[1], curr)
  

100%|██████████| 27749/27749 [00:14<00:00, 1933.58it/s]


In [184]:
print(RenderTree(root))

BKK2022
├── งานอํานวยการและบริหารสํานักงานเขต 10350780 บาท
│   ├── 1. งบบุคลากร 5716350 บาท
│   │   ├── 1.1 เงินเดือน 3648750 บาท
│   │   │   ├── 01101-2 อัตราเดิม 10 อัตรา 3178850 บาท
│   │   │   ├── 01102-1 เงินเลื่อนขั้น 190750 บาท
│   │   │   ├── 01106-1 เงินประจําตําแหน่ง 127200 บาท
│   │   │   ├── 01107-1 เงินค่าตอบแทนรายเดือนของข้าราชการ 148200 บาท
│   │   │   ├── 01108-1 เงินเพิ่มการครองชีพชั่วคราวของข้าราชการ 2900 บาท
│   │   │   └── 01109-1 เงินช่วยเหลือค่าครองชีพของข้าราชการ 850 บาท
│   │   ├── 1.2 ค่าจ้างประจํา 1706650 บาท
│   │   │   ├── 01201-1 อัตราเดิม 9 อัตรา 1586700 บาท
│   │   │   ├── 01202-1 เงินเพิ่มค่าจ้างประจํา 72400 บาท
│   │   │   ├── 01205-1 เงินเพิ่มการครองซีพชั่วคราวของลูกจ้างประจํา 33050 บาท
│   │   │   └── 01206-1 เงินช่วยเหลือค่าครองชีพของลูกจ้างประจํา 16500 บาท
│   │   ├── 1.3 ค่าจ้างชั่วคราว 312000 บาท
│   │   │   ├── 02101-1 ค่าจ้างชั่วคราว 3 อัตรา 231600 บาท
│   │   │   ├── 02102-1 เงินเพิ่มการครองซีพชั่วคราวของลูกจ้างชั่วคราว 28400 บาท
│   │   │   └─

In [95]:
job = ''
bulletFlag = False
stack = []
entries = []
entry = []

for i, group in fpl_group:
    i[0]
    lineText = group.text.values
    if not i[2]:
        continue
    if ' ' in group.iloc[0].text:
        bullet = get_patern_of_bullet(group.iloc[0].text.split(' ')[0])
    else:
        bullet = get_patern_of_bullet(group.iloc[0].text)

    if lineText[0].startswith('งาน') and group.iloc[0].line_num <= 2:
        if (lineText[-1] == 'บาท'):
            lineText = lineText[:-2]
        job = ' '.join(lineText)
        continue
    elif bullet[1]:
        bulletFlag = True

    if bulletFlag:
        entry += group.index.to_list()
        if lineText[-1] == 'บาท':

            bulletFlag = False
            entry_bullet = get_patern_of_bullet(
                df.loc[entry[0]].text.split(' ')[0])

            e = (entry, entry_bullet[1])
            if len(stack) == 0:
                stack.append(e)
            elif e[1] > stack[-1][1]:
                stack.append(e)
            elif e[1] == stack[-1][1]:
                entries.append(
                    {'job': job, 'nodes': [s[0] for s in stack]})
                stack.pop()
                stack.append(e)
            else:
                entries.append(
                    {'job': job, 'nodes': [s[0] for s in stack]})
                while len(stack) > 0 and e[1] <= stack[-1][1]:
                    stack.pop()
                stack.append(e)
            entry = []
        
entries.append(
    {'job': job, 'nodes': [s[0] for s in stack]})

('65026.pdf', 22, 0)
('65026.pdf', 22, 1)
('65026.pdf', 22, 2)
('65026.pdf', 22, 3)
('65026.pdf', 22, 4)
('65026.pdf', 22, 5)
('65026.pdf', 22, 6)
('65026.pdf', 22, 7)
('65026.pdf', 22, 8)
('65026.pdf', 22, 9)
('65026.pdf', 22, 10)
('65026.pdf', 22, 11)
('65026.pdf', 22, 12)
('65026.pdf', 22, 13)
('65026.pdf', 22, 14)
('65026.pdf', 22, 15)
('65026.pdf', 22, 16)
('65026.pdf', 22, 17)
('65026.pdf', 22, 18)
('65026.pdf', 22, 19)
('65026.pdf', 22, 20)
('65026.pdf', 22, 21)
('65026.pdf', 22, 22)
('65026.pdf', 22, 23)
('65026.pdf', 22, 24)
('65026.pdf', 22, 25)
('65026.pdf', 22, 26)
('65026.pdf', 22, 27)
('65026.pdf', 22, 28)
('65026.pdf', 22, 29)
('65026.pdf', 22, 30)
('65026.pdf', 22, 31)
('65026.pdf', 22, 32)
('65026.pdf', 23, 0)
('65026.pdf', 23, 1)
('65026.pdf', 23, 2)
('65026.pdf', 23, 3)
('65026.pdf', 23, 4)
('65026.pdf', 23, 5)
('65026.pdf', 23, 6)
('65026.pdf', 23, 7)
('65026.pdf', 23, 8)
('65026.pdf', 23, 9)
('65026.pdf', 23, 10)
('65026.pdf', 23, 11)
('65026.pdf', 23, 12)
('65026.

KeyboardInterrupt: 